# [Module 2.1] Create Dataset Group

![Fig.2.1.dataset-group-structure](static/imgs/Fig.2.1.dataset-group-structure.png)

MovieLens 데이터 세트에서 수집된 데이터를 기반으로, 영화에 대한 추천 모델을 작성하는 법을 안내합니다. 목표는 특정 사용자를 기반으로 하는 영화를 추천하는 것입니다.<br>
이 노트북에서는 전체적으로 데이타를 준비하는 단계입니다. 그러기 위해서 아래와 같은 작업을 수행 합니다. <br>

* IAM Role 생성 및 권한 할당
    - Personalize 서비스가 사용할 역할을 생성 및 권한을 할당 합니다.
* 데이타 세트 그룹 생성 (DatasetGroup)    
* 데이타 스키마 생성
* 데이타 세트 생성 (Dataset)
* 데이타 Import (S3 --> Personalize 서비스로 다운로드)

---
이 노트북을 모두 실행하는데 걸리는 시간은 약 20분 소요 됩니다.


In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdate
from botocore.exceptions import ClientError

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [3]:
suffix = str(np.random.uniform())[4:9]

In [4]:
%store -r

## Personalize Service의 S3 접근 권한 및 Personalize IAM Role 생성

Personalize Service 는 해당 S3 버킷에 접근하기 위해서 권한

또한, Amazon Personalize는 특정 작업들을 실행할 권한을 갖기 위해, AWS에서 역할을 맡을 수 있는 기능이 필요합니다. 
예를 들어 Personalize 는 S3에 접근을 해야 합니다. 그래서 이를 위한 역할이 필요하고, 이 역할은 S3 접근 권한이 필요 합니다.

In [5]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*",
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'NHPAHXGVY2DST7Q6',
  'HostId': 'mKQR4rOijIeDTxGg0KGqLPIOCHw0xOtR8S6QClatE73nEa+r5ZohLb6K16X9e8UW+M4fN9afeK0=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'mKQR4rOijIeDTxGg0KGqLPIOCHw0xOtR8S6QClatE73nEa+r5ZohLb6K16X9e8UW+M4fN9afeK0=',
   'x-amz-request-id': 'NHPAHXGVY2DST7Q6',
   'date': 'Sun, 26 Feb 2023 02:00:50 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [6]:
suffix

'76427'

In [7]:
iam = boto3.client("iam")

# Personalize 서비스가 이용할 role을 만들기 위한 assume_role_policy 생성
role_name = "PersonalizeRoleDemo" + suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

# Personalize 서비스가 이용할 role 생성
create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# 위에서 생성한 role에 AmazonPersonalizeFullAccess 권한 추가
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# 위에서 생성한 role에 AmazonS3FullAccess 권한 추가
iam.attach_role_policy(
    RoleName=role_name,    
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::057716757052:role/PersonalizeRoleDemo76427


## 데이터 세트 그룹 생성 및 대기

Personalize에서 가장 큰 단위는 **데이터 세트 그룹(Dataset Group)** 이며, 이렇게 하면 데이터, 이벤트 추적기(event tracker), 솔루션(solution) 및 캠페인(campaign)이 분리됩니다. 공통의 데이터 수집을 공유하는 것들을 그룹화합니다. 원하는 경우 아래 그룹명을 자유롭게 변경해 주세요.

In [8]:
suffix

'76427'

### 데이터 세트 그룹 생성

In [9]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "Movielens-dataset-group-" + suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:ap-northeast-2:057716757052:dataset-group/Movielens-dataset-group-76427",
  "ResponseMetadata": {
    "RequestId": "ced792fa-1401-4b41-9775-5088a8a23ce1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:01:51 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "113",
      "connection": "keep-alive",
      "x-amzn-requestid": "ced792fa-1401-4b41-9775-5088a8a23ce1"
    },
    "RetryAttempts": 0
  }
}


#### 데이터 세트 그룹이 활성화 상태가 될 때까지 대기

아래의 모든 항목에서 Dataset Group을 사용하려면 활성화(active)가 되어야 합니다. 아래 셀을 실행하고 DatasetGroup: ACTIVE로 변경될 때까지 기다려 주세요.

In [10]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


## 스키마 생성

Personalize가 데이터를 이해하는 방법의 핵심 구성 요소는 아래 정의 된 스키마(schema)에서 비롯됩니다. 이 설정은 CSV 파일을 통해 제공된 데이터를 요약하는 방법을 Personalize 서비스에 알려줍니다. 열(column)과 유형(type)은 위에서 만든 파일의 내용과 일치합니다.

In [11]:
interaction_schema_name="DEMO-interaction-schema-"+suffix

schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_VALUE",
            "type": "float"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        { 
            "name": "EVENT_TYPE",
            "type": "string"
        },
    ],
    "version": "1.0"
}


create_schema_response = personalize.create_schema(
    name = interaction_schema_name,
    schema = json.dumps(schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-northeast-2:057716757052:schema/DEMO-interaction-schema-76427",
  "ResponseMetadata": {
    "RequestId": "a47f7cb2-5f05-4207-8cb2-8295211e7243",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:02:06 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "100",
      "connection": "keep-alive",
      "x-amzn-requestid": "a47f7cb2-5f05-4207-8cb2-8295211e7243"
    },
    "RetryAttempts": 0
  }
}


In [12]:
item_schema_name="DEMO-item-schema-"+suffix

item_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
    {
        "name": "ITEM_ID",
        "type": "string"
    },
    {
        "name": "GENRE",
        "type": "string",
        "categorical": True
    }
    ],
    "version": "1.0"
}

create_metadata_schema_response = personalize.create_schema(
    name = item_schema_name,
    schema = json.dumps(item_schema)
)

item_schema_arn = create_metadata_schema_response['schemaArn']
print(json.dumps(create_metadata_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-northeast-2:057716757052:schema/DEMO-item-schema-76427",
  "ResponseMetadata": {
    "RequestId": "6de8816d-7b6a-4687-b0cb-1373e74d5a82",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:02:06 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "93",
      "connection": "keep-alive",
      "x-amzn-requestid": "6de8816d-7b6a-4687-b0cb-1373e74d5a82"
    },
    "RetryAttempts": 0
  }
}


## 데이터 세트 생성

그룹 다음으로 생성할 것은 실제 데이터 세트입니다. 아래의 코드 셀을 실행하여 데이터 세트을 생성해 주세요.

### Interaction 데이터 세트 생성

In [13]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "DEMO-interaction-dataset-" + suffix,
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interaction_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-northeast-2:057716757052:dataset/Movielens-dataset-group-76427/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "e283f0a2-a4e8-4a1f-8cdc-9ee876285cdd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:02:06 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "115",
      "connection": "keep-alive",
      "x-amzn-requestid": "e283f0a2-a4e8-4a1f-8cdc-9ee876285cdd"
    },
    "RetryAttempts": 0
  }
}


###  ITEM 데이터 세트 생성 

In [14]:
dataset_type = "ITEMS"
create_item_dataset_response = personalize.create_dataset(
    name = "DEMO-item-dataset-" + suffix,
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_schema_arn,
  
)

item_dataset_arn = create_item_dataset_response['datasetArn']
print(json.dumps(create_item_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-northeast-2:057716757052:dataset/Movielens-dataset-group-76427/ITEMS",
  "ResponseMetadata": {
    "RequestId": "8214f4ff-0260-4035-a0c5-12a12bcb3354",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:02:06 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "108",
      "connection": "keep-alive",
      "x-amzn-requestid": "8214f4ff-0260-4035-a0c5-12a12bcb3354"
    },
    "RetryAttempts": 0
  }
}


#### 데이터 세트 생성 위해서 잠시 대기(에: 20초)

In [15]:
import time

time.sleep(20)

### 데이터 세트 Import

이전에는 정보를 저장하기 위해 데이터 세트 그룹 및 데이터 세트를 생성했으므로, 
이제는 모델 구축을 위해 S3에서 Amazon Personalize로 데이터를 로드하는 import job을 실행합니다.

#### Interaction 데이터 세트 Import Job 생성

In [16]:
role_arn

'arn:aws:iam::057716757052:role/PersonalizeRoleDemo76427'

In [18]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "demo-interaction-dataset-import-" + suffix,
    datasetArn = interaction_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, warm_train_interaction_filename)
    },
    roleArn = role_arn
)

interation_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-northeast-2:057716757052:dataset-import-job/demo-interaction-dataset-import-76427",
  "ResponseMetadata": {
    "RequestId": "95e90ae3-c22d-460a-8265-aeb337a385a9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:03:13 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "130",
      "connection": "keep-alive",
      "x-amzn-requestid": "95e90ae3-c22d-460a-8265-aeb337a385a9"
    },
    "RetryAttempts": 0
  }
}


#### 아이템 데이터 세트 Import Job 생성

In [19]:
create_item_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "demo-item-dataset-import-" + suffix,
    datasetArn = item_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, items_filename)
    },
    roleArn = role_arn
)

item_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))


{
  "datasetImportJobArn": "arn:aws:personalize:ap-northeast-2:057716757052:dataset-import-job/demo-interaction-dataset-import-76427",
  "ResponseMetadata": {
    "RequestId": "95e90ae3-c22d-460a-8265-aeb337a385a9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:03:13 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "130",
      "connection": "keep-alive",
      "x-amzn-requestid": "95e90ae3-c22d-460a-8265-aeb337a385a9"
    },
    "RetryAttempts": 0
  }
}


### 데이터 세트 Import job이 활성화 상태가 될 때까지 대기

Import job이 완료되기까지 시간이 걸립니다. 아래 코드 셀의 출력 결과가 DatasetImportJob: ACTIVE가 될 때까지 기다려 주세요.

In [20]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interation_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 39.4 ms, sys: 8.74 ms, total: 48.1 ms
Wall time: 4min


In [21]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = item_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)



DatasetImportJob: ACTIVE


## Review

위의 코드를 사용하여 데이타 세트 그룹, 데이타 세트, 데이타 세트 Import까지를 수행 하였습니다. 다음 단계는 이를 기반으로 솔류션(모델)을 생성하는 단계를 진행 합니다.


## 다음 노트북에 대한 참고 사항

다음 실습에 필요한 몇 가지 값들이 있습니다. 아래 셀을 실행하여 저장한 후, 다음 주피터 노트북에서 그대로 사용할 수 있습니다.

In [22]:
%store dataset_group_arn
%store interaction_dataset_arn
%store item_dataset_arn
%store interaction_schema_arn
%store item_schema_arn
%store role_name
%store role_arn
%store suffix

Stored 'dataset_group_arn' (str)
Stored 'interaction_dataset_arn' (str)
Stored 'item_dataset_arn' (str)
Stored 'interaction_schema_arn' (str)
Stored 'item_schema_arn' (str)
Stored 'role_name' (str)
Stored 'role_arn' (str)
Stored 'suffix' (str)
